In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, UpSampling1D, Reshape, Dropout
from keras.models import Model, load_model
import scipy.io
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import h5py

import tensorflow as tf
import keras.backend as K

from tensorflow.keras.layers import Input, SimpleRNN, Dense, Bidirectional ,Dropout, Add, multiply, GlobalAvgPool1D, Reshape, concatenate, UpSampling1D, Flatten, MaxPooling1D, BatchNormalization, average, Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from math import*
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.ops.math_ops import reduce_prod

In [ ]:
mat = scipy.io.loadmat('labels.mat')


label_lat = mat['latit_arr']
label_long = mat['longit_arr']
label_mag = mat['magnitude_arr']


label_lat = pd.DataFrame(label_lat.T)
label_long = pd.DataFrame(label_long.T)
label_mag = pd.DataFrame(label_mag.T)
# print(label_lat.T.shape)
# print(label_long.T.shape)
# print(label_mag.T.shape)

label = np.array([label_lat, label_long, label_mag, label_depth]).T
label = label.reshape(len(label[0]),3)

print(label.shape)
print(label)

In [ ]:
factor = pd.read_csv(r'dataset.csv')

In [ ]:
print(type(factor))
print(factor.shape)

In [ ]:
factor1 = np.array(factor)
factor1.shape
len(factor1)

In [ ]:
selected_rows1 = factor1[5::9]

In [ ]:
print(selected_rows1.shape)

In [ ]:
# Define a custom cosine similarity metric
def cosine_similarity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return K.sum(y_true * y_pred, axis=-1)

In [ ]:
''' 
#from tensorflow.keras.layers import *
#from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, TimeDistributed, Bidirectional

def create_RNN(hidden_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(15, input_shape=(15,10), 
                        activation='linear',return_sequences=True))
    #model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
 
demo_model = create_RNN(15,(15,10), activation=['linear', 'linear'])

demo_model.summary()
#model = Model(inputs=inp, outputs=C1)
#model.summary()
'''

In [ ]:
label_lat =label[:,0]
print(label_lat.shape)

label_long =label[:,1]
print(label_long.shape)

label_mag =label[:,2]
print(label_mag.shape)


label_lat = np.reshape(label_lat,(len(label_lat),1))
label_long = np.reshape(label_long,(len(label_long),1))
label_mag = np.reshape(label_mag,(len(label_mag),1))

min_lat = np.min(label_lat, axis=0)
max_lat = np.max(label_lat, axis=0)

min_long = np.min(label_long, axis=0)
max_long = np.max(label_long, axis=0)

min_mag = np.min(label_mag, axis=0)
max_mag = np.max(label_mag, axis=0)


print(min_lat)
print(max_lat)
print(min_long)
print(max_long)
print(min_mag)
print(max_mag)

# Normalize the labels to the [0, 1] range
label_lat_norm = (label_lat - min_lat) / (max_lat - min_lat)
print(label_lat_norm)

label_long_norm = (label_long - min_long) / (max_long - min_long)
print(label_long_norm)

label_mag_norm = (label_mag - min_mag) / (max_mag - min_mag)
print(label_mag_norm)

In [ ]:
label_norm = np.concatenate((label_lat_norm, label_long_norm, label_mag_norm), axis=1)
label_norm.shape

In [ ]:
features =  np.reshape(selected_rows1, (len(selected_rows1), 1,  selected_rows1.shape[1],1))
print(features.shape)

features_train, features_test, label_train, label_test = train_test_split( features, label_norm,  test_size=0.2, random_state=1)

In [ ]:
dataInput__train_1 = features_train[:,0:-1]
dataInput__train_2 = features_train[:,-1]

dataInput__valid_1 = features_test[:,0:-1]
dataInput__valid_2 = features_test[:,-1]

print(dataInput__train_1.shape)
print(dataInput__train_2.shape)
print(dataInput__valid_1.shape)
print(dataInput__valid_2.shape)


In [ ]:
dataInput_reshape__1 = np.reshape(dataInput__train_1,(dataInput__train_1.shape[0],1,dataInput__train_1.shape[1],1))
# print(dataInput_reshape__1.shape)

dataInput_reshape_val__1 = np.reshape(dataInput__valid_1,(dataInput__valid_1.shape[0],1,dataInput__valid_1.shape[1],1))
# print(dataInput_reshape_val__1.shape)

dataInput_reshape2__1 = np.reshape(dataInput__train_2,(dataInput__train_2.shape[0],1,1))
# print(dataInput_reshape2__1.shape)

dataInput_reshape2_val__1 = np.reshape(dataInput__valid_2,(dataInput__valid_2.shape[0],1,1))
# print(dataInput_reshape2_val__1.shape)

In [ ]:
input_sig1__1 = Input(batch_shape=(None,1,401,1))
input_sig2__1 = Input(batch_shape=(None,1,1,1))

############################################################################
##                                 CNN

conv__input_sig2__1 = Conv2D(filters=128, kernel_size=(1, 1))(input_sig2__1)

x = Conv2D(256,(3,1), activation='relu', padding='same')(input_sig1__1)
# Dropout(0.3)
# x1 = MaxPooling2D(1,3)(x)
x2 = Conv2D(64,(3,1), activation='relu', padding='same')(x)
# Dropout(0.3)
# x3 = MaxPooling2D(1,3)(x2)
x4 = Conv2D(64,(3,1), activation='relu', padding='same')(x2)
# Dropout(0.3)
x5 = Conv2D(32,(3,1), activation='relu', padding='same')(x4)
# Dropout(0.3)###


x6 = Conv2D(64,(3,1), activation='relu', padding='same')(x5)
# Dropout(0.3)

xa = Conv2D(64,(3,1), activation='relu', padding='same')(x6)#####

x7 = Conv2D(128,(3,1), activation='relu', padding='same')(xa)

x8 = concatenate([x7, conv__input_sig2__1], axis=2)

inp_bilstm = layers.Reshape((402, 128))(x8)


############################################################################
##                                 BiLSTM

C1 = Bidirectional(layers.LSTM(200, return_sequences=True))(inp_bilstm)
#C1 = Dropout(0.2)(C1)

C2 = Bidirectional(layers.LSTM(200, return_sequences=True))(C1)
#C2 = Dropout(0.2)(C2)

C3 = Bidirectional(layers.LSTM(200, return_sequences=True))(C2)
#C3 = Dropout(0.2)(C3)

C4 = Bidirectional(layers.LSTM(200, return_sequences=False))(C3)

C5 = layers.Dense(3, activation='relu')(C4)

ourmodel = Model([input_sig1__1,input_sig2__1], outputs=C5)

print(ourmodel.summary())

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


callbacks = [
    EarlyStopping(patience=20, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000000001, verbose=1),
    ModelCheckpoint('BiLSTM__zComponent_concat.h5',monitor='val_loss',mode='min',save_best_only=True)
]

ourmodel.compile(optimizer='adam', loss='mae')
history = ourmodel.fit([dataInput_reshape__1,dataInput_reshape2__1], label_train, batch_size=64, epochs=500,shuffle=True,validation_data=([dataInput_reshape_val__1,dataInput_reshape2_val__1],label_test) ,verbose=1,callbacks=callbacks)

In [ ]:
out_train= ourmodel.predict([dataInput_reshape__1,dataInput_reshape2__1])

out_test= ourmodel.predict([dataInput_reshape_val__1,dataInput_reshape2_val__1])

In [ ]:
pred0 = ((out_test[:,0])*(max_lat-min_lat))+min_lat
pred0 =np.reshape(pred0, (len(pred0), 1))
pred1 = ((out_test[:,1])*(max_long-min_long))+min_long
pred1 =np.reshape(pred1, (len(pred1), 1))
pred2 = ((out_test[:,2])*(max_mag-min_mag))+min_mag
pred2 =np.reshape(pred2, (len(pred2), 1))


pred_orig = np.concatenate([pred0,pred1,pred2], axis=1)
pred_orig

In [ ]:
plt.plot(pred_orig[:,0],measure_orig[:,0],'.')
plt.plot([measure_orig[:,0].min(), measure_orig[:,0].max()], [measure_orig[:,0].min(), measure_orig[:,0].max()], 'k--', lw=4)


plt.xlabel('Predicted Latitude', fontsize=14,fontweight="bold")
plt.ylabel('Measured Latitude', fontsize=14,fontweight="bold")

#################################################################
#################################################################
                      
plt.xticks(fontsize=14,fontweight="bold")
plt.yticks(fontsize=14,fontweight="bold")
plt.savefig("diag_lat.pdf",bbox_inches='tight')
plt.show()